In [23]:
# src/monte_carlo.py
import numpy as np

def monte_carlo_pricer(
    S0, K, T, r, sigma, option_type="call",
    n_sim=1000000, antithetic=True, seed=None
):
    """
    Monte Carlo pricer for European options under Black-Scholes.

    Parameters
    ----------
    S0 : float
        Initial stock price
    K : float
        Strike price
    T : float
        Time to maturity (in years)
    r : float
        Risk-free interest rate
    sigma : float
        Volatility
    option_type : str
        "call" or "put"
    n_sim : int
        Number of Monte Carlo simulations
    antithetic : bool
        If True, use antithetic variates
    seed : int or None
        Random seed for reproducibility

    Returns
    -------
    price : float
        Estimated option price
    se : float
        Standard error of the estimate
    """
    if seed is not None:
        np.random.seed(seed)

    # number of paths
    n_paths = n_sim // 2 if antithetic else n_sim

    # draw standard normals
    Z = np.random.randn(n_paths)

    # terminal stock price: S_T
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T)

    ST1 = S0 * np.exp(drift + diffusion * Z)

    if antithetic:
        ST2 = S0 * np.exp(drift - diffusion * Z)
        ST = np.concatenate([ST1, ST2])
    else:
        ST = ST1

    # payoff
    if option_type.lower() == "call":
        payoff = np.maximum(ST - K, 0.0)
    else:
        payoff = np.maximum(K - ST, 0.0)

    # discounted payoff
    discounted_payoff = np.exp(-r * T) * payoff

    # mean and SE
    price = np.mean(discounted_payoff)
    se = np.std(discounted_payoff, ddof=1) / np.sqrt(len(discounted_payoff))

    return price, se


# -----------------------
# Example Run
# -----------------------
if __name__ == "__main__":
    S0 = 100     # Spot
    K = 100      # Strike
    T = 1        # 1 year
    r = 0.05     # Risk-free rate
    sigma = 0.2  # Volatility

    for opt in ["call", "put"]:
        price, se = monte_carlo_pricer(S0, K, T, r, sigma, opt, n_sim=1000000, antithetic=True, seed=45)
        print(f"{opt.upper()} option price (MC): {price:.4f} ± {1.96*se:.4f} (95% CI)")


CALL option price (MC): 10.4318 ± 0.0288 (95% CI)
PUT option price (MC): 5.5610 ± 0.0169 (95% CI)
